In [1]:
# Import Dependencies
from pathlib import Path
import pandas as pd
import requests
import json
from api_key import open_weather_api_key
import numpy as np
from datetime import datetime, timedelta
import calendar
import sklearn.datasets as dta
import scipy.stats as st
import matplotlib.pyplot as plt
from scipy.stats import linregress

In [2]:
#Create a path variable to the data
cities_path = Path('Resources/32180DS0001_2001-21.xlsx')

In [3]:
# Import the data into a Pandas DataFrame
cities_data_df = pd.read_excel(cities_path, sheet_name="Table 1")
cities_data_df.head(20)

,Australian Bureau of Statistics,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41
0,"Regional population, 2021",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Released at 11.30am (Canberra time) 26 July 2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Table 1. Estimated resident population, Statis...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Change,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2011-2021,NaN,NaN,Area,Population density 2021,NaN,NaN,NaN,NaN,NaN
6,S/T code,S/T name,GCCSA code,GCCSA name,SA4 code,SA4 name,SA3 code,SA3 name,SA2 code,SA2 name,...,no.,%,NaN,km2,persons/km2,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1,New South Wales,1RNSW,Rest of NSW,101,Capital Region,10102,Queanbeyan,101021007,Braidwood,...,872,25.2,NaN,3418.4,1.3,NaN,NaN,NaN,NaN,NaN
9,1,New South Wales,1RNSW,Rest of NSW,101,Capital Region,10102,Queanbeyan,101021008,Karabar,...,-546,-6,NaN,7,1223.9,NaN,NaN,NaN,NaN,NaN


In [4]:
#Clean the dataframe
dropped_row_cities_df = cities_data_df.iloc[8:2462]
dropped_column_cities_df = dropped_row_cities_df[["Unnamed: 1","Unnamed: 9","Unnamed: 30","Unnamed: 35","Unnamed: 36"]]
cleaned_cities_df = dropped_column_cities_df.rename(columns={"Unnamed: 1": "State Name", "Unnamed: 9": "Statistical Areas Level 2 Name", "Unnamed: 30": "Population", "Unnamed: 35": "Area km2","Unnamed: 36": "Population Density (Persons/km2)"}).reset_index()
del cleaned_cities_df["index"]
cleaned_cities_df


,State Name,Statistical Areas Level 2 Name,Population,Area km2,Population Density (Persons/km2)
0,New South Wales,Braidwood,4330,3418.4,1.3
1,New South Wales,Karabar,8546,7,1223.9
2,New South Wales,Queanbeyan,11370,4.8,2387.7
3,New South Wales,Queanbeyan,5093,13,391.7
4,New South Wales,Queanbeyan West,12743,13.7,931.9
...,...,...,...,...,...
2449,Australian Capital Territory,Namadgi,67,1202.8,0.1
2450,Other Territories,Christmas Island,1716,136.1,12.6
2451,Other Territories,Cocos (Keeling) Islands,602,13.7,43.9
2452,Other Territories,Jervis Bay,310,67.2,4.6


In [5]:
#Getting latitude and longitude data 

cities_url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

# Build partial query URL
query_url = f"{cities_url}appid={open_weather_api_key}&units={units}&q="

# set up lists to hold reponse info
lat = []
lon = []

# Loop through the list of cities and perform a request for data on each
for index, row in cleaned_cities_df.iterrows():
    city = row["Statistical Areas Level 2 Name"]
    try:
        response = requests.get(query_url + city).json()
        lat.append(response['coord']['lat'])
        lon.append(response['coord']['lon'])
    except:
        lat.append(np.nan)
        lon.append(np.nan)



In [7]:
#Add Latitude and Longitude to the dataframe
cleaned_cities_df["Latitude"] = lat
cleaned_cities_df["Longitude"] = lon
cleaned_cities_df


,State Name,Statistical Areas Level 2 Name,Population,Area km2,Population Density (Persons/km2),Latitude,Longitude
0,New South Wales,Braidwood,4330,3418.4,1.3,41.2650,-88.2123
1,New South Wales,Karabar,8546,7,1223.9,-35.3753,149.2308
2,New South Wales,Queanbeyan,11370,4.8,2387.7,-35.3500,149.2333
3,New South Wales,Queanbeyan,5093,13,391.7,-35.3500,149.2333
4,New South Wales,Queanbeyan West,12743,13.7,931.9,NaN,NaN
...,...,...,...,...,...,...,...
2449,Australian Capital Territory,Namadgi,67,1202.8,0.1,NaN,NaN
2450,Other Territories,Christmas Island,1716,136.1,12.6,-10.5000,105.6667
2451,Other Territories,Cocos (Keeling) Islands,602,13.7,43.9,NaN,NaN
2452,Other Territories,Jervis Bay,310,67.2,4.6,-35.1333,150.7000


In [8]:
#Clean new dataframe
cities_data_final_df = cleaned_cities_df.dropna(how="any").reset_index()
del cities_data_final_df['index']
cities_data_final_df


,State Name,Statistical Areas Level 2 Name,Population,Area km2,Population Density (Persons/km2),Latitude,Longitude
0,New South Wales,Braidwood,4330,3418.4,1.3,41.2650,-88.2123
1,New South Wales,Karabar,8546,7,1223.9,-35.3753,149.2308
2,New South Wales,Queanbeyan,11370,4.8,2387.7,-35.3500,149.2333
3,New South Wales,Queanbeyan,5093,13,391.7,-35.3500,149.2333
4,New South Wales,Googong,6245,6.9,902.9,-35.4177,149.2334
...,...,...,...,...,...,...,...
1795,Australian Capital Territory,Molonglo,0,2.2,0,-35.2998,149.0402
1796,Australian Capital Territory,Wright,3806,1.3,2993.1,45.1833,-93.9502
1797,Other Territories,Christmas Island,1716,136.1,12.6,-10.5000,105.6667
1798,Other Territories,Jervis Bay,310,67.2,4.6,-35.1333,150.7000


In [9]:
#Set up a list of time frame from Jan 2023 to Jan 2024 mothly
start_date = '2021-01-06'
end_date = '2022-01-01'
time = np.arange(np.datetime64(start_date), np.datetime64(end_date),timedelta(days=30))
ux_time = time.astype('datetime64[s]').astype('int')
ux_time

array([1609891200, 1612483200, 1615075200, 1617667200, 1620259200,
       1622851200, 1625443200, 1628035200, 1630627200, 1633219200,
       1635811200, 1638403200])

In [13]:
#Create empty list for storing data
co = []
o3 =[]
no2 =[]
so2 = []
pm2_5 = []
pm10 = []


#Loop through all cities/row in dataframe
for index, row in cleaned_cities_df.iterrows():
    lat = row["Latitude"]
    lon =row["Longitude"]
    #Loop through all time in time frame
    for i in range(len(ux_time)):
        start = ux_time[i]
        end = start+ 1
        daily_url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start}&end={end}&appid={open_weather_api_key}"
        data = requests.get(daily_url).json()
        try: 
            #Add data to respective lists
            co.append(data["list"][0]["components"]['co'])
            o3.append(data["list"][0]["components"]['o3'])
            no2.append(data["list"][0]["components"]['no2'])
            so2.append(data["list"][0]["components"]['so2'])
            pm2_5.append(data["list"][0]["components"]['pm2_5'])
            pm10.append(data["list"][0]["components"]['pm10'])    
        except (KeyError, IndexError):
            co.append(np.nan)
            o3.append(np.nan)
            no2.append(np.nan)
            so2.append(np.nan)
            pm2_5.append(np.nan)
            pm10.append(np.nan) 

    #Calculate average data
    clean_co = [x for x in co if not np.isnan(x)]
    clean_o3 = [x for x in o3 if not np.isnan(x)]
    clean_no2 = [x for x in no2 if not np.isnan(x)]
    clean_so2 = [x for x in so2 if not np.isnan(x)]
    clean_pm2_5 = [x for x in pm2_5 if not np.isnan(x)]
    clean_pm10 = [x for x in pm10 if not np.isnan(x)]

    average_co = sum(clean_co) / len(clean_co)
    average_o3 = sum(clean_o3) / len(clean_o3)
    average_no2 = sum(clean_no2) / len(clean_no2)
    average_so2 = sum(clean_so2) / len(clean_so2)
    average_pm2_5 = sum(clean_pm2_5) / len(clean_pm2_5)
    average_pm10 = sum(clean_pm10) / len(clean_pm10)
    
    #Add data to dataframe
    cities_data_final_df.loc[index, "Average CO (μg/m3)"] = average_co
    cities_data_final_df.loc[index, "Average O3 (μg/m3)"] = average_o3
    cities_data_final_df.loc[index, "Average NO2 (μg/m3)"] = average_no2
    cities_data_final_df.loc[index, "Average S02 (μg/m3)"] = average_so2
    cities_data_final_df.loc[index, "Average PM2.5 (μg/m3)"] = average_pm2_5
    cities_data_final_df.loc[index, "Average PM10 (μg/m3)"] = average_pm10


In [73]:
#Drop NaN values
cities_data_final_df = cities_data_final_df.dropna(how="any")
cities_data_final_df

,State Name,Statistical Areas Level 2 Name,Population,Area km2,Population Density (Persons/km2),Latitude,Longitude,Average CO (μg/m3),Average O3 (μg/m3),Average NO2 (μg/m3),Average S02 (μg/m3),Average PM2.5 (μg/m3),Average PM10 (μg/m3)
0,New South Wales,Braidwood,4330,3418.4,1.0,41.2650,-88.2123,283.442500,72.566667,8.796667,1.959167,8.870000,9.447500
1,New South Wales,Karabar,8546,7,"1,224.0",-35.3753,149.2308,255.904583,55.647917,6.565417,1.588750,6.675000,7.647500
2,New South Wales,Queanbeyan,11370,4.8,"2,388.0",-35.3500,149.2333,246.725278,50.008333,5.821667,1.465278,5.943333,7.047500
3,New South Wales,Queanbeyan,5093,13,392.0,-35.3500,149.2333,242.135625,47.188542,5.449792,1.403542,5.577500,6.747500
4,New South Wales,Googong,6245,6.9,903.0,-35.4177,149.2334,242.135625,47.188542,5.449792,1.403542,5.577500,6.747500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,Australian Capital Territory,Molonglo,0,2.2,0.0,-35.2998,149.0402,253.008652,42.785339,8.280131,3.799508,4.958023,7.525435
1796,Australian Capital Territory,Wright,3806,1.3,"2,993.0",45.1833,-93.9502,252.990538,42.776613,8.279230,3.797958,4.956432,7.523417
1797,Other Territories,Christmas Island,1716,136.1,13.0,-10.5000,105.6667,253.006920,42.763641,8.282959,3.797872,4.955646,7.522579
1798,Other Territories,Jervis Bay,310,67.2,5.0,-35.1333,150.7000,252.988835,42.754944,8.282057,3.796325,4.954060,7.520567


In [78]:
# Save the clean DataFrame to a CSV file without the index.
cities_data_final_df.to_csv("Output/air_quality.csv", index=False)